<a href="https://colab.research.google.com/github/FatemehRanjbar/Image_process/blob/main/Persian_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>